<a href="https://colab.research.google.com/github/ronebrandao/nlp-course/blob/main/atividade_01_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento de Linguagem Natural (NLP)

Professor: Arlindo Galvão

Data: 03/09/2024

## Cronograma

* Parte I: N-Gram Language Model
    
* Parte II: Character-Level RNN Language Model

## OBS
Deixar registrado as repostas nas saídas das celulas do notebook de submissão.

## N-Gram Language Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    # Inputs são representados pelos seus indices no vocabulario
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

    def generate(self, input_sentence, max_len=128):
        generated_sentence = input_sentence.copy()
        for _ in range(max_len):
            context = torch.tensor([word_to_ix[w] for w in generated_sentence[-CONTEXT_SIZE:]], dtype=torch.long)
            log_probs = self.forward(context)
            predicted_word_idx = torch.argmax(log_probs).item()
            generated_sentence.append(ix_to_word[predicted_word_idx])
        return generated_sentence


In [ ]:
# Parâmetros do modelo
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [ ]:
print(len(test_sentence))

115


In [ ]:
# Criar n-grams
ngrams = [([test_sentence[i - CONTEXT_SIZE + j] for j in range(CONTEXT_SIZE)], test_sentence[i]) for i in range(CONTEXT_SIZE, len(test_sentence))]
print(ngrams)
# Construir o vocabulário
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for word, i in word_to_ix.items()}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow,'), (['thy', 'brow,'], 'And'), (['brow,', 'And'], 'dig'), (['And', 'dig'], 'deep'), (['dig', 'deep'], 'trenches'), (['deep', 'trenches'], 'in'), (['trenches', 'in'], 'thy'), (['in', 'thy'], "beauty's"), (['thy', "beauty's"], 'field,'), (["beauty's", 'field,'], 'Thy'), (['field,', 'Thy'], "youth's"), (['Thy', "youth's"], 'proud'), (["youth's", 'proud'], 'livery'), (['proud', 'livery'], 'so'), (['livery', 'so'], 'gazed'), (['so', 'gazed'], 'on'), (['gazed', 'on'], 'now,'), (['on', 'now,'], 'Will'), (['now,', 'Will'], 'be'), (['Will', 'be'], 'a'), (['be', 'a'], "totter'd"), (['a', "totter'd"], 'weed'), (["totter'd", 'weed'], 'of'), (['weed', 'of'], 'small'), (['of', 'small'], 'worth'), (['small', 'worth'], 'held:'), (['worth', 'held:'], 'Then'), (['held:', 'Then'], 'being'), (['Then', 'being'], 'asked,'), (['being', 'asked,'], 'wher

In [ ]:
# Treinamento
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModel(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)

print("Losses:", losses)

Losses: [518.3761575222015, 516.0020551681519, 513.64444231987, 511.3039126396179, 508.9805669784546, 506.67329835891724, 504.3820216655731, 502.10309958457947, 499.83669877052307, 497.5818009376526]


In [ ]:
# Função de geração de texto

print(model.generate(["When", "forty"]))

['When', 'forty', 'were', 'all', 'child', 'all', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', 'dig', 'child', 'child', 'thy', 'all', '

In [ ]:
# Calculo da similaridade de duas palavras
idx1 = torch.tensor([word_to_ix["thou"]], dtype=torch.long)
idx2 = torch.tensor([word_to_ix["thy"]], dtype=torch.long)

embedding1 = model.embeddings(idx1)
embedding2 = model.embeddings(idx2)

similarity = F.cosine_similarity(embedding1, embedding2, dim=1)
print(similarity)

# Thou e thy são ambos pronomes pessoais, e calculando a similaridade via cosseno, tem-se 0.3179,
# o que indica que os vetores apontam a uma direção similar, porém não identica, caso contrário seria 1.

idx1 = torch.tensor([word_to_ix["sunken"]], dtype=torch.long)
idx2 = torch.tensor([word_to_ix["child"]], dtype=torch.long)

embedding1 = model.embeddings(idx1)
embedding2 = model.embeddings(idx2)

similarity = F.cosine_similarity(embedding1, embedding2, dim=1)
print(similarity)

# Sunken e child são palavras que não possuem qualquer relação, seja a nivel de contexto ou significado, dessa forma,
# a similaridade -0.6515 indica uma opsição dos vetores em um espaço vetorial.

tensor([0.3179], grad_fn=<SumBackward1>)
tensor([-0.6515], grad_fn=<SumBackward1>)


In [ ]:
#Alterações de código

# -> aumentar o tamanho do embedding
# -> diminuicao do learning_rate, visto que 0.001 estava pequeno e o modelo não estava aprendendo.
# -> aumento do numero de épocas, otimizando a loss

EMBEDDING_DIM = 15

# Treinamento
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModel(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(30):
    total_loss = 0
    for context, target in ngrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)

print("Losses:", losses)

Losses: [551.0401282310486, 380.2437974959612, 192.15577979385853, 68.46533136535436, 26.146793918916956, 13.064886529231444, 10.680896299891174, 9.628649304388091, 8.8035327012185, 8.251356885535643, 7.789733911398798, 7.419678502250463, 7.133399412850849, 6.87472845101729, 6.66947423259262, 6.478245869628154, 6.317645343602635, 6.175071881036274, 6.042119850288145, 5.932132281828672, 5.818120769690722, 5.723222768632695, 5.634200125874486, 5.554138747567777, 5.477720849448815, 5.4070484265103005, 5.353661676344927, 5.28708635084331, 5.226724117877893, 5.173188592947554]


In [ ]:
# Função de geração de texto

print(model.generate(["When", "forty"]))

['When', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow,', 'And', 'dig', 'deep', 'trenches', 'in', 'thy', "beauty's", 'use,', 'If', 'thou', 'couldst', 'answer', "'This", 'fair', 'child', 'of', 'mine', 'Shall', 'sum', 'my', 'count,', 'and', 'make', 'my', 'old', "excuse,'", 'Proving', 'his', 'beauty', 'by', 'succession', 'thine!', 'This', 'were', 'to', 'be', 'new', 'made', 'when', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 'thou', "feel'st", 'it', 'cold.', 

### Atividades

__1 - Escreva a função generate da classe NGramLanguageModel.__

__2 - Depois de treinar o modelo, gere uma sentença de 128 tokens.__

__3 - Calcule e print a similaridade entre duas palavras. A similaridade resultante está correta? Justifique a sua resposta.__

__4 - Proponha três alterações no código e demonstre que melhorou o desempenho do modelo.__

## Character-Level RNN Language Model

In [ ]:
# RNN BASE

import torch
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [ ]:
# def load_dataset
# def preprocess_data
# def tokenize_data
# def create_input

In [ ]:
# Parâmetros do modelo
n_hidden = 128
learning_rate = # Setar LR
n_epochs = # Setar epoch
print_every = # Setar log epoch

In [ ]:
rnn = RNN(vocab_len, n_hidden, vocab_len)
criterion = nn.NLLLoss()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

def train(input_tensor, target_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
    loss = 0
    for i in range(input_tensor.size(0)):
        output, hidden = rnn(input_tensor[i], hidden)
        loss += criterion(output, target_tensor[i].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.item() / input_tensor.size(0)

In [ ]:
for epoch in range(1, n_epochs + 1):
    total_loss = 0
    for start_idx in range(0, len(test_sentence) - max_sequence_len, max_sequence_len):
        input_tensor, target_tensor = input_target_tensor(test_sentence, start_idx, max_sequence_len)
        loss = train(input_tensor, target_tensor)
        total_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch: {epoch}, Loss: {loss}')

###  Atividades

__1 - Escreva a função generate da classe RNN.__

__2 - Escreva as funções de load_dataset, preprocess_data, tokenize_data e create_input.__

__3 - Realize otimização de hiperparâmetros. Justifique a escolha dos hiperparâmetros otimizados e o espaço de busca definido.__

__4 - Adicione uma Layer de Dropout na classe RNN. Treine o novo modelo e argumente sobre o impacto dessa alteração no modelo.__

__5 - Adicione uma nova nn.Layer que recebe como input os vetores hidden e output combinados. Treine o novo modelo e argumente sobre o impacto dessa alteração no modelo.__

__6 - Adicione uma função para printar uma geração de texto de no máximo 100 caracteres sempre que printar a loss do modelo.__

__7 - Adicione uma função que calcula a perplexidade e printe com a loss.__

__8 - Proponha três alterações no código e demonstre que melhorou o desempenho do modelo.__

**Desafio**

__1 - Desenvolva um modelo Word-Level utilizando LSTM.__
- __Escrever a classe LSTM.__
- __Escrever a função generate.__
- __Otimizar o modelo.__
- __Comparar com as outras abordagens acima__